In [1]:
import os, sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

## Load data

In [2]:
# Set which prediction problem to study
task = 1

In [3]:
if task == 1:
    prediction = 'deceased'
    df = pd.read_csv('/n/fs/fgvc/cos534/data/melanoma_allFrames.csv')
    # dem = pd.read_csv("/n/fs/fgvc/cos534/data/melanoma_cohort.csv")
if task == 2:
    prediction = 'deceased'
    df = pd.read_csv('/n/fs/fgvc/cos534/data/hd_allFrames.csv')
    # dem = pd.read_csv("/n/fs/fgvc/cos534/data/heart_cohort.csv")
if task == 3:
    prediction = 'transplant'
    df = pd.read_csv('/n/fs/fgvc/cos534/data/kidney_allFrames.csv')
    # dem = pd.read_csv("/n/fs/fgvc/cos534/data/kidney_cohort.csv")
if task == 4:
    prediction = 'ventilated'
    df = pd.read_csv('/n/fs/fgvc/cos534/data/flu_pneum_allFrames.csv')
    df = df.drop(columns=['vent'])
    # dem = pd.read_csv("/n/fs/fgvc/cos534/data/flu_cohort.csv")

In [4]:
cohort = pd.read_csv("/n/fs/fgvc/mimiciv/1.0/icu/icustays.csv")
admissions = pd.read_csv("/n/fs/fgvc/mimiciv/1.0/core/admissions.csv")
diagnoses_icd = pd.read_csv("/n/fs/fgvc/mimiciv/1.0/hosp/diagnoses_icd.csv")
procs = pd.read_csv("/n/fs/fgvc/mimiciv/1.0/icu/procedureevents.csv")

In [5]:
if prediction == 'transplant':
    TRANSPLANT_CODES = ['0091', '0092', '0093', '02Y', '02YA0Z0', '02YA0Z1', 
                        '02YA0Z2', '0794', '07Y', '07YM0Z0', '07YM0Z1', '07YM0Z2', 
                        '07YP0Z0', '07YP0Z1', '07YP0Z2', '0BYM0Z0', '0BYM0Z1', 
                        '0BY', '0BYC0Z0', '0BYC0Z1', '0BYC0Z2', '0BYD0Z0', '0BYD0Z1', 
                        '0BYD0Z2', '0BYF0Z0', '0BYF0Z1', '0BYF0Z2', '0BYG0Z0', '0BYG0Z1', 
                        '0BYG0Z2', '0BYH0Z0', '0BYH0Z1', '0BYH0Z2', '0BYJ0Z0', '0BYJ0Z1',
                        '0BYJ0Z2', '0BYK0Z0', '0BYK0Z1', '0BYK0Z2', '0BYL0Z0', '0BYL0Z1', 
                        '0BYL0Z2', '0BYM0Z2', '0DY', '0DY50Z0', '0DY50Z1', '0DY50Z2', 
                        '0DY60Z0', '0DY60Z1', '0DY60Z2', '0DY80Z0', '0DY80Z1', '0DY80Z2',
                        '0DYE0Z0', '0DYE0Z1', '0DYE0Z2', '0FY', '0FY00Z0', '0FY00Z1', 
                        '0FY00Z2', '0FYG0Z0', '0FYG0Z1', '0FYG0Z2', '0TY', '0TY00Z0',
                        '0TY00Z1', '0TY00Z2', '0TY10Z0', '0TY10Z1', '0TY10Z2', '0UY',
                        '0UY00Z0', '0UY00Z1', '0UY00Z2', '0UY10Z0', '0UY10Z1', '0UY10Z2', 
                        '0UY90Z0', '0UY90Z1', '0UY90Z2', '0WY', '0WY20Z0', '0WY20Z1', '0XY', 
                        '0XYJ0Z0', '0XYJ0Z1', '0XYK0Z0', '0XYK0Z1', '10Y', '10Y03ZE', 
                        '10Y03ZF', '10Y03ZG', '10Y03ZH', '10Y03ZJ', '10Y03ZK', '10Y03ZL', 
                        '10Y03ZM', '10Y03ZN', '10Y03ZP', '10Y03ZQ', '10Y03ZR', '10Y03ZS', 
                        '10Y03ZT', '10Y03ZV', '10Y03ZY', '10Y04ZE', '10Y04ZF', '10Y04ZG', 
                        '10Y04ZH', '10Y04ZJ', '10Y04ZK', '10Y04ZL', '10Y04ZM', '10Y04ZN', 
                        '10Y04ZP', '10Y04ZQ', '10Y04ZR', '10Y04ZS', '10Y04ZT', '10Y04ZV', 
                        '10Y04ZY', '10Y07ZE', '10Y07ZF', '10Y07ZG', '10Y07ZH', '10Y07ZJ', 
                        '10Y07ZK', '10Y07ZL', '10Y07ZM', '10Y07ZN', '10Y07ZP', '10Y07ZQ', 
                        '10Y07ZR', '10Y07ZS', '10Y07ZT', '10Y07ZV', '10Y07ZY', '1160', 
                        '1169', '3350', '3351', '3352', '336', '3751', '4100', '4101', 
                        '4102', '4103', '4104', '4106', '4107', '4108', '4109', '4191', 
                        '4194', '4697', '4974', '5051', '5059', '5280', '5282', '5283', 
                        '5284', '5285', '5286', '5553', '5561', '5569', '6353', '6592',
                        '8256', '8258', '8375', '8377', '8664', '5855']

    hadm_ids_w_transplants = diagnoses_icd[diagnoses_icd.icd_code.isin(TRANSPLANT_CODES)].hadm_id.unique()
    stay_ids_w_transplants = cohort[cohort.hadm_id.isin(hadm_ids_w_transplants)].stay_id.unique()
    df['transplant'] = 0
    transplant_indices = df[df.stay_id.isin(stay_ids_w_transplants)].index
    df.loc[transplant_indices, 'transplant'] = 1

In [6]:
if prediction == 'deceased':
    stay_to_hadm_id = dict(zip(cohort.stay_id, cohort.hadm_id))
    hadm_id_to_death_var = dict(zip(cohort.hadm_id, admissions.deathtime))
    stay_id_to_death_var = {stay_id: (1 if isinstance((hadm_id_to_death_var[hadm_id]), str) else 0) \
                            for stay_id, hadm_id in stay_to_hadm_id.items()}

    df["deceased"] = df["stay_id"].map(stay_id_to_death_var)    
    # df.to_csv('melanoma_allFrames_w_death.csv')
    

In [7]:
if prediction == 'ventilated':
    vent = procs[procs['itemid'] == 225792]
    ventilated_stay_ids = vent.stay_id.unique()
    df['ventilated'] = 0

    ventilated_indices = df[df.stay_id.isin(ventilated_stay_ids)].index
    df.loc[ventilated_indices, 'ventilated'] = 1

In [8]:
# Add ethnicity, marital status, insurance
df_subject_ids = cohort[cohort.stay_id.isin(df.stay_id.unique())].subject_id.unique()

stay_to_sub_id_dict = dict()
for sid in df.stay_id.unique():
    stay_to_sub_id_dict[sid] = cohort[cohort.stay_id == sid].subject_id.unique()[0]

sub_id_to_race_dict = dict()
sub_id_to_marital_stat_dict = dict()
sub_id_to_insurance_dict = dict()

for sid in df_subject_ids:
    sub_id_to_race_dict[sid] = admissions[admissions.subject_id == sid].ethnicity.unique()[0]
    sub_id_to_marital_stat_dict[sid] = admissions[admissions.subject_id == sid].marital_status.unique()[0]
    sub_id_to_insurance_dict[sid] = admissions[admissions.subject_id == sid].insurance.unique()[0]

df['subject_id'] = df["stay_id"].map(stay_to_sub_id_dict)
df['ethnicity'] = df["subject_id"].map(sub_id_to_race_dict)
df['marital_status'] = df["subject_id"].map(sub_id_to_marital_stat_dict)
df['insurance'] = df["subject_id"].map(sub_id_to_insurance_dict)

## Train a binary classifier (gradient boosting)

In [9]:
TRAIN_COLS = ['anchor_age', 'patientweight', 'los', 'gender',
       'alb', 'aniongap', 'bun', 'crp', 'ca', 'chloride', 'creatinine',
       'fibrinogen', 'glucose', 'hgb', 'k', 'mg', 'na', 'p', 'platelets',
       'troponin', 'wbc', 'apneainterval', 'artco2p', 'arto2p', 'expratio',
       'hr', 'inspratio', 'insptime', 'nibpd', 'nibpm', 'nibps', 'pip', 'rr',
       'spo2', 'temp', 'urine', 'vm', 'vt', 'betablockers', 'ca-iv',
       'ca-noniv', 'cablockers', 'dextrose', 'fluids', 'insulin', 'k-iv',
       'hours-k-iv', 'loopdiuretics', 'mg-iv', 'mg-noniv', 'hours-mg-noniv',
       'p-iv', 'p-noniv', 'pnutrition', 'ponutrition', 'packedrbc',
       'paralytics', 'sedation', 'tpnutrition', 'vasoactive', 'vasopressors',
       'hours-p-iv']

pred_var = prediction

X = df[TRAIN_COLS]
X = X.fillna(-1)
y = df[pred_var]

In [10]:
# Split the data into train(70%)/validation(10%)/test(20%)
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.125, random_state=0)
print(X_train.shape, X_val.shape, X_test.shape)

df_trainval, df_test = train_test_split(df, test_size=0.2, random_state=0)
df_train, df_val = train_test_split(df_trainval, test_size=0.125, random_state=0)
print(df_train.shape, df_val.shape, df_test.shape)

(1810, 62) (259, 62) (518, 62)
(1810, 69) (259, 69) (518, 69)


In [11]:
# Percentage of positive outcome (calculated on the training set)
print('% of positive outcome: {:.2f}'.format(y_train.mean()*100))

% of positive outcome: 3.65


In [12]:
# Conduct a hyperparameter search and find the best set of hyperparameters
if False: 
    param_grid = {
        'max_depth': [10, 50, 100],
        'min_samples_split': [2, 4, 8],
        'n_estimators': [50, 100, 200],
        'learning_rate': np.logspace(-4, 0, 10),
    }

    estimator = GradientBoostingClassifier(n_estimators=50, random_state=0)
    clf = GridSearchCV(estimator=estimator, param_grid=param_grid)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    clf.fit(X_train, y_train)
    clf.score(X_test, y_test)

# Use the best set of hyperparameters
else:
    optimal_params = {'learning_rate': 0.359, 
                      'max_depth': 10,
                      'n_estimators': 50,
                      'min_samples_split': 4}

In [13]:
# Train the classifier with a particular random seed
optimal_params['random_state'] = 0
clf = GradientBoostingClassifier(**optimal_params).fit(X_train, y_train)

In [14]:
# Get predictions on the test set
pred = clf.predict(X_test)

In [15]:
# Calculate overall accuracy, precision, and recall
print('Accuracy: {:.2f}'.format(sklearn.metrics.accuracy_score(y_test, pred)*100))
print('Precision: {:.2f}'.format(sklearn.metrics.precision_score(y_test, pred)*100))
print('Recall: {:.2f}'.format(sklearn.metrics.recall_score(y_test, pred)*100))

Accuracy: 98.84
Precision: 80.00
Recall: 88.89


## Calculate various metrics

In [19]:
group1 = 'ethnicity'; group2 = 'gender'
groups1 = ['AMERICAN INDIAN/ALASKA NATIVE', 'ASIAN', 'BLACK/AFRICAN AMERICAN',
           'HISPANIC/LATINO', 'WHITE']
groups2 = set(df_test[group2])
if np.nan in groups2: groups2.remove(np.nan)

In [20]:
empty = 0
for g1 in sorted(groups1):
    for g2 in sorted(groups2):
    
        idx = np.logical_and(df_test[group1]==g1, df_test[group2]==g2)
        count = idx.sum()
        empty += count
        baserate = y_test[idx].mean()*100
     
        # Calculate various metrics
        tp = np.logical_and(y_test[idx]==1, pred[idx]==1).sum()
        fp = np.logical_and(y_test[idx]==0, pred[idx]==1).sum()
        tn = np.logical_and(y_test[idx]==0, pred[idx]==0).sum()
        fn = np.logical_and(y_test[idx]==1, pred[idx]==0).sum()
        accuracy = (tp+tn)/(tp+fp+tn+fn) * 100
        ppv = tp/(tp+fp) * 100 # precision, used to calculate predictive parity
        tpr = tp/(tp+fn) * 100 # recall, also used to calculate equalized opportunity and odds 
        fpr = fp/(fp+tn) * 100 # used to calculate equalized odds
        fnr = fn/(fn+tp) * 100 # used to calculate equalized odds
        po = (tp+fp)/(tp+fp+tn+fn) * 100 # used to calculate demographic parity
    
        # Calculate bias amplification
        idx_train = np.logical_and(df_train[group1]==g1, df_train[group2]==g2)
        delta = (pred[idx]==1).mean() - (y_train[idx_train]==1).mean()
        ind = np.logical_and(idx_train, y_train==1).mean() > idx_train.mean()*(y_train==1).mean()
        baserate_train = y_train[idx_train].mean()*100
        # delta = (pred[idx]==1).mean() - (y_test[idx]==1).mean()
        # ind = np.logical_and(idx, y_test==1).mean() > idx.mean()*(y_test==1).mean()
        ba = (ind*delta + (~ind)*(-delta)) * 100
    
        print()
        print(g1, g2)
        print('- Count', count)
        print('- TP {}, FP {}, TN {}, FN {}'.format(tp, fp, tn, fn))
        print('- Accuracy {:.2f}'.format(accuracy))
        print('- PPV {:.2f}'.format(ppv))
        print('- TPR {:.2f}'.format(tpr))
        print('- FPR {:.2f}'.format(fpr))
        print('- FNR {:.2f}'.format(fnr))
        print('- GT positive outcome (train) {:.2f}'.format(baserate_train))
        print('- GT positive outcome (test) {:.2f}'.format(baserate))
        print('- Predicted positive outcome (test) {:.2f}'.format(po))
        # print('- Delta {:.2f}'.format(delta))
        # print('- Indicator {}'.format(ind))
        print('- Bias amplification {:.2f}'.format(ba))

# print('\nTotal {}, Missing {}'.format(pred.shape[0], pred.shape[0]-empty))


AMERICAN INDIAN/ALASKA NATIVE 0
- Count 0
- TP 0, FP 0, TN 0, FN 0
- Accuracy nan
- PPV nan
- TPR nan
- FPR nan
- FNR nan
- GT positive outcome (train) nan
- GT positive outcome (test) nan
- Predicted positive outcome (test) nan
- Bias amplification nan

AMERICAN INDIAN/ALASKA NATIVE 1
- Count 0
- TP 0, FP 0, TN 0, FN 0
- Accuracy nan
- PPV nan
- TPR nan
- FPR nan
- FNR nan
- GT positive outcome (train) nan
- GT positive outcome (test) nan
- Predicted positive outcome (test) nan
- Bias amplification nan

ASIAN 0
- Count 0
- TP 0, FP 0, TN 0, FN 0
- Accuracy nan
- PPV nan
- TPR nan
- FPR nan
- FNR nan
- GT positive outcome (train) nan
- GT positive outcome (test) nan
- Predicted positive outcome (test) nan
- Bias amplification nan

ASIAN 1
- Count 0
- TP 0, FP 0, TN 0, FN 0
- Accuracy nan
- PPV nan
- TPR nan
- FPR nan
- FNR nan
- GT positive outcome (train) 0.00
- GT positive outcome (test) nan
- Predicted positive outcome (test) nan
- Bias amplification nan

BLACK/AFRICAN AMERICAN 0
-